In [5]:
import numpy as np
import joblib

In [ ]:
def compute_burnout_index(responses):
    responses = np.array(responses, dtype=float)

    ee_questions = [0, 1, 2, 7, 12, 13, 15, 19]
    dp_questions = [4, 9, 10, 14, 21]
    pa_questions = [3, 6, 8, 11, 16, 17, 18, 20]

    ee_score = responses[ee_questions].sum()
    dp_score = responses[dp_questions].sum()
    ee_score += (6 - responses[5])
    pa_score = responses[pa_questions].sum()

    burnout_index = np.sqrt(
        ((ee_score / 54) ** 2 +
         (dp_score / 30) ** 2 +
         ((1 - pa_score / 48)) ** 2) / 3
    )
    return float(burnout_index)

def load_model_and_scaler(
    model_path: str = MODEL_PATH,
    scaler_path: str = SCALER_PATH
):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    return model, scaler

def predict_burnout(responses, model=None, scaler=None):
    responses = np.array(responses, dtype=float)

    if responses.shape[0] != 23:
        raise ValueError(f"Ожидалось 23 ответа, получено {responses.shape[0]}")

    burnout_index = compute_burnout_index(responses)

    features = np.concatenate([responses, [burnout_index]]).reshape(1, -1)

    if model is None or scaler is None:
        model, scaler = load_model_and_scaler()

    X_scaled = scaler.transform(features)
    proba = model.predict_proba(X_scaled)[0]
    label = int(model.predict(X_scaled)[0])

    classes = list(model.classes_)
    prob_dict = {int(cls): float(p) for cls, p in zip(classes, proba)}

    return {
        "label": label,
        "label_name": CLASS_NAMES.get(label, f"class_{label}"),
        "probabilities": prob_dict,
        "burnout_index": float(burnout_index),
    }

In [8]:
MODEL_PATH = "burnout_model.pkl"
SCALER_PATH = "burnout_scaler.pkl"

CLASS_NAMES = {
    0: "Нет выгорания",
    1: "Умеренное выгорание",
    2: "Сильное выгорание",
}

if __name__ == "__main__":
    example_responses = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4]
    result = predict_burnout(example_responses)

    print("Ответы:", example_responses)
    print("Интегральная метрика (burnout_index):", f"{result['burnout_index']:.4f}")
    print("Предсказанный класс:", result["label"], "-", result["label_name"])
    print("Вероятности классов:")
    for cls, p in result["probabilities"].items():
        print(f"  класс {cls} ({CLASS_NAMES.get(cls, '')}): {p:.4f}")


Ответы: [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4]
Интегральная метрика (burnout_index): 0.4986
Предсказанный класс: 1 - Умеренное выгорание
Вероятности классов:
  класс 0 (Нет выгорания): 0.0001
  класс 1 (Умеренное выгорание): 0.9999
  класс 2 (Сильное выгорание): 0.0000


/opt/venvs/seaborn-env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
